In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense

2023-03-25 16:59:54.165985: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
# load training dataset
with open('datasets/train.dataset.6mer.npy', 'rb') as open_file:
    df = np.load(open_file)
df = pd.DataFrame(df)

In [4]:
six_mers = pd.read_table('datasets/6mer_columns.txt')
df.columns = six_mers
df.columns = [col[0] for col in df.columns]

In [5]:
labels = pd.read_csv('datasets/train_labels.csv')
df['genome_label'] = labels
df.head()

,AAAAAT,AAAAAG,AAAAAC,AAAATA,AAAATT,AAAATG,AAAATC,AAAAGA,AAAAGT,AAAAGG,...,CCAGGG,CCACGG,CCTAGG,CCCAGG,CCGAGG,CCCCGG,CCGCGG,CCCGGG,IGNORE,genome_label
0,0.002470,0.004528,0.003292,0.000823,0.003704,0.002880,0.001646,0.001646,0.001646,0.001646,...,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,staphylococcus_aureus
1,0.001818,0.002857,0.002077,0.001558,0.003635,0.002338,0.002338,0.001039,0.001818,0.001039,...,0.000519,0.0,0.0,0.000519,0.000260,0.0,0.0,0.0,0.0,staphylococcus_aureus
2,0.003702,0.003084,0.001234,0.001851,0.002468,0.003084,0.003084,0.001851,0.001234,0.000617,...,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,staphylococcus_aureus
3,0.001102,0.002756,0.003584,0.001378,0.003307,0.002481,0.002481,0.001102,0.001654,0.001378,...,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,staphylococcus_aureus
4,0.004318,0.003534,0.002748,0.001701,0.003534,0.002356,0.002224,0.002617,0.003271,0.001440,...,0.000000,0.0,0.0,0.000262,0.000131,0.0,0.0,0.0,0.0,staphylococcus_aureus


In [ ]:
df['genome_label'].value_counts()

In [ ]:
df['genome_label'].value_counts().sum()

In [ ]:
len(df['genome_label'].unique())

In [ ]:
df[df.isna().any(axis=1)]

There are 30 different pathogens and one decoy label. There are 2080 features and 505536 samples, as well as no missing values, in the training set.

In [6]:
# subsample with random seed for reproducibility
# sample_size = df.genome_label.value_counts().to_frame().min().values[0]
df_train = df.groupby('genome_label').apply(lambda x: x.sample(200, random_state=4)) # use all the samples

In [ ]:
# eda
pd.plotting.scatter_matrix(df_train, figsize=[8, 8], s=150, marker='D')
# sns.pairplot(df_train)
# plt.show()

Feature selection: Remove highly correlated features

In [8]:
mask = np.triu(np.ones_like(df_train.corr(), dtype=bool)) # remove duplicate upper triangle correlation values
corr_matrix = df_train.corr().abs() # create positive correlation matrix
tri_df = corr_matrix.mask(mask) # create and apply upper triangle mask
to_drop = [c for c in tri_df.columns if any(tri_df[c] > 0.8)] # drop columns that meet threshold

/var/folders/61/_1tynjrd31972whvq0rhfvmc0000gn/T/ipykernel_53229/3399044558.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  mask = np.triu(np.ones_like(df_train.corr(), dtype=bool)) # remove duplicate upper triangle correlation values
/var/folders/61/_1tynjrd31972whvq0rhfvmc0000gn/T/ipykernel_53229/3399044558.py:2: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr_matrix = df_train.corr().abs() # create positive correlation matrix


In [9]:
df_train.drop(columns=to_drop, inplace=True)

Label processing: Encode categorical labels

In [10]:
df_train = pd.get_dummies(df_train, columns=['genome_label'], prefix='', prefix_sep='')
df_train

AAAAAT    AAAAAC    AAAATA    AAAATC  \
genome_label                                                            
acinetobacter_baumannii 50199  0.001087  0.001812  0.001812  0.000362   
                        49240  0.002443  0.001504  0.001880  0.001128   
                        49495  0.006226  0.004761  0.001831  0.000732   
                        49212  0.003307  0.002316  0.002316  0.002645   
                        49659  0.000734  0.001101  0.001101  0.001101   
...                                 ...       ...       ...       ...   
yersinia_enterocolitica 74537  0.003098  0.001430  0.001667  0.000476   
                        74937  0.000773  0.001160  0.001160  0.001547   
                        73572  0.002253  0.001072  0.001072  0.000751   
                        74908  0.001391  0.000556  0.002226  0.001670   
                        73823  0.000000  0.000669  0.000669  0.001339   

                                 AAAAGA    AAAAGT    AAAAGG    AAAAGC  \
genome_label                                                            
acinetobacter_baumannii 50199  0.000725  0.000362  0.001087  0.001087   
                        49240  0.001316  0.002068  0.000940  0.000564   
                        49495  0.002930  0.003296  0.001099  0.001465   
                        49212  0.002645  0.001323  0.000661  0.000661   
                        49659  0.001652  0.001835  0.000183  0.000367   
...                                 ...       ...       ...       ...   
yersinia_enterocolitica 74537  0.000715  0.000476  0.000715  0.000476   
                        74937  0.000902  0.000902  0.000387  0.000773   
                        73572  0.001180  0.000643  0.000643  0.000214   
                        74908  0.000835  0.000835  0.001391  0.000278   
                        73823  0.002678  0.000000  0.000000  0.000669   

                                 AAAACA    AAAACT  ...  \
genome_label                                       ...   
acinetobacter_baumannii 50199  0.002537  0.002174  ...   
                        49240  0.001316  0.001504  ...   
                        49495  0.001831  0.001831  ...   
                        49212  0.003307  0.002645  ...   
                        49659  0.000367  0.001101  ...   
...                                 ...       ...  ...   
yersinia_enterocolitica 74537  0.002382  0.001191  ...   
                        74937  0.000773  0.000515  ...   
                        73572  0.000965  0.001180  ...   
                        74908  0.000556  0.001670  ...   
                        73823  0.000000  0.000669  ...   

                               staphylococcus_pseudintermedius  \
genome_label                                                     
acinetobacter_baumannii 50199                                0   
                        49240                                0   
                        49495                                0   
                        49212                                0   
                        49659                                0   
...                                                        ...   
yersinia_enterocolitica 74537                                0   
                        74937                                0   
                        73572                                0   
                        74908                                0   
                        73823                                0   

                               staphylococcus_pyogenes  \
genome_label                                             
acinetobacter_baumannii 50199                        0   
                        49240                        0   
                        49495                        0   
                        49212                        0   
                        49659                        0   
...                                                ...   
yersinia_enterocolitica 74537                        0 

In [32]:
df_train.iloc[:, 2025:].values

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1]], dtype=uint8)

Dimensionality reduction using PCA (We can try UMAP?)

In [11]:
pca = PCA(n_components=50, svd_solver='full')
pca.fit_transform(df_train.iloc[:, :2024].values)


array([[-0.004815  , -0.00411713, -0.0013386 , ...,  0.0005676 ,
        -0.00022195, -0.00050624],
       [-0.01187492,  0.00110101,  0.00152402, ...,  0.00109905,
         0.00027716, -0.00052278],
       [-0.01218702,  0.00010893, -0.00149988, ..., -0.00120418,
        -0.00022736, -0.00039659],
       ...,
       [-0.00396328, -0.00249141,  0.00054212, ...,  0.00051752,
        -0.00050016,  0.00052863],
       [-0.00106742, -0.0036179 , -0.00099751, ...,  0.00051922,
        -0.00064791,  0.00019041],
       [ 0.00600982, -0.00499511, -0.0002203 , ..., -0.00033927,
         0.00099336,  0.00226937]])

In [12]:
X_train = pca.fit_transform(df_train.iloc[:, :2024].values) # filter by explained variance ratio?
y_train = df_train.iloc[:, 2025:].values
X_train.shape, y_train.shape

((6200, 50), (6200, 31))

Use neural network as model

In [13]:
# define the model
n_inputs = 50 # no. of features
n_outputs = 31 # no. of classes
model = Sequential()
model.add(Dense(20, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu')) # add more layers
model.add(Dense(n_outputs, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam')

2023-03-25 17:14:40.713657: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [14]:
# fit model
model.fit(X_train, y_train, verbose=0, epochs=100) # do cross-validation to optimise
model.save('model.h5')

In [15]:
def precision_per_patient(patient_id, preds):
    df_true = pd.read_csv(f'datasets/validation/patient{patient_id}_labels.txt')
    tp, fp, tp_labels = 0, 0, df_true['true_label'].shape[0]
    print(f'my prediction(s) for patient {patient_id}: {preds}')
    truth = df_true['true_label'].values
    print(f'true pathogen(s): {truth}')
    #if don't predict any pathogen, it means there is only decoy in the test dataset (your prediction)
    if len(preds) == 0:
        preds = ['decoy']
    for item in np.unique(preds):
        if item in df_true['true_label'].values:
            tp+=1
        else:
            fp+=1
    #you have to predict all labels correctly, but you are penalized for any false positive
    return tp/(tp_labels+fp)

In [16]:
# load trained model
model = load_model('model.h5')

In [27]:
threshold = 0.99
classes = df['genome_label'].unique()

all_precision = []
for patient_id in range(1, 11):
    print(f'predicting for patient {patient_id}')
    
    with open(f'datasets/validation/patient{patient_id}.6mer.npy', 'rb') as read_file:
        df_test = np.load(read_file)
    df_test = pd.DataFrame(df_test)
    X_test = pca.fit_transform(df_test.iloc[:, :2024].values)
        
    # predict test set
    y_pred = model.predict(X_test) # each column is the probability that the row is a particular pathogen

    final_predictions = [classes[lab] for lab in np.unique([np.argmax(item) for item in y_pred if np.max(item) >= threshold])]
    
    # my pathogens detected, decoy will be ignored
    final_predictions = [item for item in final_predictions if item !='decoy']
    
    precision = precision_per_patient(patient_id, final_predictions)
    print(f'precision: {precision}')
    all_precision.append(precision)

predicting for patient 1
313/313 [==============================] - 0s 609us/step
my prediction(s) for patient 1: ['salmonella_enterica_paratyphi', 'campylobacter_jejuni']
true pathogen(s): ['staphylococcus_aureus']
precision: 0.0
predicting for patient 2
313/313 [==============================] - 0s 414us/step
my prediction(s) for patient 2: []
true pathogen(s): ['staphylococcus_pyogenes']
precision: 0.0
predicting for patient 3
313/313 [==============================] - 0s 422us/step
my prediction(s) for patient 3: ['campylobacter_jejuni']
true pathogen(s): ['burkholderia_pseudomallei' 'corynebacterium_ulcerans']
precision: 0.0
predicting for patient 4
313/313 [==============================] - 0s 421us/step
my prediction(s) for patient 4: ['staphylococcus_pyogenes', 'listeria_monocytogenes']
true pathogen(s): ['pseudomonas_aeruginosa']
precision: 0.0
predicting for patient 5
313/313 [==============================] - 0s 426us/step
my prediction(s) for patient 5: ['staphylococcus_pyo

In [ ]:
# performance per patient and its final average
print([f'patient {c}: {item}' for c, item in enumerate(all_precision, start=1)])
print(f'avg: {np.mean(all_precision)}')